In [2]:
import numpy as np
import pandas as pd


In [3]:
True_News=pd.read_csv("Datasets\True.csv")
Fake_News=pd.read_csv("Datasets\Fake.csv")

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\F'
<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:2: SyntaxWarning: invalid escape sequence '\F'
C:\Users\user\AppData\Local\Temp\ipykernel_22596\3738873834.py:1: SyntaxWarning: invalid escape sequence '\T'
  True_News=pd.read_csv("Datasets\True.csv")
C:\Users\user\AppData\Local\Temp\ipykernel_22596\3738873834.py:2: SyntaxWarning: invalid escape sequence '\F'
  Fake_News=pd.read_csv("Datasets\Fake.csv")


**Lets go to make ready our data**

In [4]:
True_News.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
True_News["label"]=1
Fake_News["label"]=0

In [44]:
Fake_News = Fake_News.drop_duplicates(subset='text')
news=pd.concat([True_News,Fake_News],axis=0)

In [45]:
news.tail()

,title,text,subject,date,label
22698,The White House and The Theatrics of ‘Gun Cont...,21st Century Wire says All the world s a stage...,US_News,"January 7, 2016",0
22699,Activists or Terrorists? How Media Controls an...,Randy Johnson 21st Century WireThe majority ...,US_News,"January 7, 2016",0
22700,"BOILER ROOM – No Surrender, No Retreat, Heads ...",Tune in to the Alternate Current Radio Network...,US_News,"January 6, 2016",0
22701,Federal Showdown Looms in Oregon After BLM Abu...,21st Century Wire says A new front has just op...,US_News,"January 4, 2016",0
22702,A Troubled King: Chicago’s Rahm Emanuel Desper...,21st Century Wire says It s not that far away....,US_News,"January 2, 2016",0


In [46]:
news=news.drop(['title','subject','date'],axis=1)

In [47]:
news.head()

,text,label
0,WASHINGTON (Reuters) - The head of a conservat...,1
1,WASHINGTON (Reuters) - Transgender people will...,1
2,WASHINGTON (Reuters) - The special counsel inv...,1
3,WASHINGTON (Reuters) - Trump campaign adviser ...,1
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,1


In [48]:
news=news.sample(frac=1)

In [49]:
news.reset_index(inplace=True)

In [50]:
news=news.drop(['index'],axis=1)

In [51]:
news.head()

,text,label
0,OTTAWA (Reuters) - U.S. Treasury Secretary Ste...,1
1,WASHINGTON (Reuters) - The chairman of the U.S...,1
2,It s no secret that nobody likes Ted Cruz.. He...,0
3,Barack Obama s reckless desire to open our bor...,0
4,PARIS (Reuters) - An economic blockade on Qata...,1


**Clean the texts**

In [52]:
import re
def clean(text):
    text=text.lower()
    text = re.sub(r'https?://\S+|www\.\S+|<.*?>|[^a-z\s]', ' ', text)
    return re.sub(r'\s+', ' ', text).strip()
news["text"]=news["text"].apply(clean)

In [53]:
news["text"][1]

'washington reuters the chairman of the u s house of representatives intelligence committee flatly denied on monday that there had been a wiretap on president donald trump s trump tower in new york but said it was possible other surveillance was used against trump let me be clear we know there was not a wiretap on trump tower however it s still possible that other surveillance activities were used against president trump and his associates republican representative devin nunes said in his opening statement at a hearing on russia s involvement in the u s presidential election nunes said numerous current and former u s officials had leaked potentially classified information and that his committee intended to identify them to bring them to justice'

In [54]:
pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [55]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return ' '.join(filtered)
news["text"]=news["text"].apply(remove_stopwords)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [56]:
news['text'][2]

'secret nobody likes ted cruz arrogant self important elitist believes speaks god face one punchable planet screechy voice makes cringe disgust starts one righteous rants help hope light fixture falls head leaving forever incapable speaking donald trump kidding said everyone congress hates ted cruz cruz refuses deal compromise within party nevermind across aisle cruz presidency would nothing short disaster country today perspective today know whiny toddler ted cruz past somehow new idea hated according article mother jones apparently prominent aide george w bush campaign could barely contain asked discuss cruz worked campaign policy shop person described cruz hyper arrogant widely despised emphasized pervasive dislike cruz within bush ranks nothing ideology cruz noted never objected bush call compassionate conservatism immigration reform national education standards one campaign regarded ideologue problem simple personality ted cruz took role ultra conservative nutbar teahadist moderat

**apply to vectorization and train_test_split**

In [57]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
X_train, X_test, y_train, y_test = train_test_split(news["text"], news["label"], test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


**Apply Random forest Classifer**

In [58]:
from sklearn.ensemble import RandomForestClassifier
RFC=RandomForestClassifier()
RFC.fit(X_train_vec,y_train)

RandomForestClassifier()

In [64]:
from sklearn.metrics import accuracy_score
y_pred = RFC.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.9956270096463022


In [63]:
from sklearn.metrics import accuracy_score

y_train_pred = RFC.predict(X_train_vec)
train_acc = accuracy_score(y_train, y_train_pred)
print("Train Accuracy:", train_acc)

Train Accuracy: 0.9999678425571599


In [66]:
def output(n):
    if n==0:
        return "Its Fake news"
    else:
        return "Its Real news"

def manualtesting(text):
    cleaned = clean(text)
    cleaned = remove_stopwords(cleaned)
    vect_text = vectorizer.transform([cleaned])  
    prediction = RFC.predict(vect_text)[0]
    return output(prediction)



In [74]:
print(manualtesting(Fake_News['text'][10]))


Its Fake news


In [80]:
import pickle
pickle.dump(RFC,open("Fake_News_Detection_Model.pkl", "wb"))